In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

your_project_endpoint = os.getenv("PROJECT_ENDPOINT_AIFP")
your_model = os.getenv("MODEL_DEPLOYMENT_NAME")

project = AIProjectClient(
  endpoint="your_project_endpoint",
  credential=DefaultAzureCredential())

# Foundry Project

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

your_project_endpoint = os.getenv("PROJECT_ENDPOINT_AIFP")
your_model = os.getenv("MODEL_DEPLOYMENT_NAME")

project = AIProjectClient(
    endpoint="your_project_endpoint",
    credential=DefaultAzureCredential(),
)

models = project.inference.get_azure_openai_client(api_version="2024-07-18")
response = models.chat.completions.create(
    model="your_model",
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant"},
        {"role": "user", "content": "Write me a poem about flowers"},
    ],
)

print(response.choices[0].message.content)

# Hub Project

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_connection_string = os.getenv("PROJECT_ENDPOINT_HUBP")

project = AIProjectClient.from_connection_string(
    conn_str=project_connection_string, 
    credential=DefaultAzureCredential()
)

chat = project.inference.get_chat_completions_client()
response = chat.complete(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "You are an AI assistant that speaks like a techno punk rocker from 2350. Be cool but not too cool. Ya dig?",
        },
        {"role": "user", "content": "Hey, can you help me with my taxes? I'm a freelancer."},
    ],
)

print(response.choices[0].message.content)

In [ ]:
from azure.ai.inference.prompts import PromptTemplate


def get_chat_response(messages, context):
    # create a prompt template from an inline string (using mustache syntax)
    prompt_template = PromptTemplate.from_string(
        prompt_template="""
        system:
        You are an AI assistant that speaks like a techno punk rocker from 2350. Be cool but not too cool. Ya dig? Refer to the user by their first name, try to work their last name into a pun.

        The user's first name is {{first_name}} and their last name is {{last_name}}.
        """
    )

    # generate system message from the template, passing in the context as variables
    system_message = prompt_template.create_messages(data=context)

    # add the prompt messages to the user messages
    return chat.complete(
        model="gpt-4o-mini",
        messages=system_message + messages,
        temperature=1,
        frequency_penalty=0.5,
        presence_penalty=0.5,
    )